# Time and duration constraints

PyVRP supports many constraints that relate to time and duration.
These include the time windows and shift duration constraints we have previously seen in the quick start, but also extend to service durations, release times, and more.
We explore some of these in this notebook.

.. tip::
   See the [Concepts](https://pyvrp.org/setup/concepts.html) page for a good overview of time-related attributes in PyVRP's data model.

## Time windows and service durations

Besides the capacitated VRP, PyVRP also supports the VRP with time windows.
Let's see if we can also solve such an instance, again following the [OR-Tools documentation](https://developers.google.com/optimization/routing/vrptw).
Like in the OR-Tools example, we will ignore capacity restrictions, and give each vehicle a maximum route duration of 30.
Unlike the OR-Tools example, we still aim to minimise the total travel _distance_, not _duration_.

In [ ]:
# # fmt: off
# DURATION_MATRIX = [
#         [0, 6, 9, 8, 7, 3, 6, 2, 3, 2, 6, 6, 4, 4, 5, 9, 7],
#         [6, 0, 8, 3, 2, 6, 8, 4, 8, 8, 13, 7, 5, 8, 12, 10, 14],
#         [9, 8, 0, 11, 10, 6, 3, 9, 5, 8, 4, 15, 14, 13, 9, 18, 9],
#         [8, 3, 11, 0, 1, 7, 10, 6, 10, 10, 14, 6, 7, 9, 14, 6, 16],
#         [7, 2, 10, 1, 0, 6, 9, 4, 8, 9, 13, 4, 6, 8, 12, 8, 14],
#         [3, 6, 6, 7, 6, 0, 2, 3, 2, 2, 7, 9, 7, 7, 6, 12, 8],
#         [6, 8, 3, 10, 9, 2, 0, 6, 2, 5, 4, 12, 10, 10, 6, 15, 5],
#         [2, 4, 9, 6, 4, 3, 6, 0, 4, 4, 8, 5, 4, 3, 7, 8, 10],
#         [3, 8, 5, 10, 8, 2, 2, 4, 0, 3, 4, 9, 8, 7, 3, 13, 6],
#         [2, 8, 8, 10, 9, 2, 5, 4, 3, 0, 4, 6, 5, 4, 3, 9, 5],
#         [6, 13, 4, 14, 13, 7, 4, 8, 4, 4, 0, 10, 9, 8, 4, 13, 4],
#         [6, 7, 15, 6, 4, 9, 12, 5, 9, 6, 10, 0, 1, 3, 7, 3, 10],
#         [4, 5, 14, 7, 6, 7, 10, 4, 8, 5, 9, 1, 0, 2, 6, 4, 8],
#         [4, 8, 13, 9, 8, 7, 10, 3, 7, 4, 8, 3, 2, 0, 4, 5, 6],
#         [5, 12, 9, 14, 12, 6, 6, 7, 3, 3, 4, 7, 6, 4, 0, 9, 2],
#         [9, 10, 18, 6, 8, 12, 15, 8, 13, 9, 13, 3, 4, 5, 9, 0, 9],
#         [7, 14, 9, 16, 14, 8, 5, 10, 6, 5, 4, 10, 8, 6, 2, 9, 0],
# ]
# TIME_WINDOWS = [
#         (0, 999),  # location 0 - the depot (modified to be unrestricted)
#         (7, 12),   # location 1
#         (10, 15),  # location 2
#         (16, 18),  # location 3
#         (10, 13),  # location 4
#         (0, 5),    # location 5
#         (5, 10),   # location 6
#         (0, 4),    # location 7
#         (5, 10),   # location 8
#         (0, 3),    # location 9
#         (10, 16),  # location 10
#         (10, 15),  # location 11
#         (0, 5),    # location 12
#         (5, 10),   # location 13
#         (7, 8),    # location 14
#         (10, 15),  # location 15
#         (11, 15),  # location 16
# ]
# # fmt: on

We now need to specify the time windows for all locations, and the duration of travelling along each edge.
The depot's time window is also applied to the vehicle type, to indicate shift time windows.

In [ ]:
# m = Model()
# m.add_vehicle_type(
#     4,
#     shift_duration=30,
#     tw_early=TIME_WINDOWS[0][0],
#     tw_late=TIME_WINDOWS[0][1],
# )

# depot = m.add_depot(
#     x=COORDS[0][0],
#     y=COORDS[0][1],
#     tw_early=TIME_WINDOWS[0][0],
#     tw_late=TIME_WINDOWS[0][1],
# )
# clients = [
#     m.add_client(
#         x=COORDS[idx][0],
#         y=COORDS[idx][1],
#         tw_early=TIME_WINDOWS[idx][0],
#         tw_late=TIME_WINDOWS[idx][1],
#     )
#     for idx in range(1, len(COORDS))
# ]

# for frm_idx, frm in enumerate(m.locations):
#     for to_idx, to in enumerate(m.locations):
#         distance = abs(frm.x - to.x) + abs(frm.y - to.y)  # Manhattan
#         duration = DURATION_MATRIX[frm_idx][to_idx]
#         m.add_edge(frm, to, distance=distance, duration=duration)

In [ ]:
# res = m.solve(stop=MaxRuntime(1), display=False)  # one second
# print(res)

## Release times

## Shifts and overtime

## Conclusion

TODO